In [ ]:
dirpath = '{{ genome_h5_dirpath }}'
job_type = '{{ job_type }}'

In [ ]:
import os

files = os.listdir(dirpath)
if job_type == 'sort':
    string_filter = '.genome.h5'
    dataset_name = 'id_trajectory'
elif job_type == 'shim':
    string_filter = '.h5'
    dataset_name = 'id_trajectory_shimmed'
h5_files = [file for file in files if string_filter in file]
h5_filepaths = [os.path.join(dirpath, file) for file in h5_files]

In [ ]:
import math

import h5py
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-whitegrid')

# Plot x trajectory and angle

In [ ]:
X_HARDCODED_DATA_INDICES = [0, 2]
X_HARDCODED_TITLES = ['X trajectory\n', 'X angle\n']

figs_axes = []
for data_index, plot_title in zip(X_HARDCODED_DATA_INDICES, X_HARDCODED_TITLES):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 10), sharex=True, sharey=True)
    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.grid(False)
    plt.title(plot_title, fontsize=20)
    plt.xlabel('s', labelpad=5, fontsize=15)
    plt.ylabel('x', labelpad=30, fontsize=15)
    # as the y axis is shared, only the 0th subplot needs to be edited, ie, the left-most one
    plt.setp(ax[0].get_yticklabels(), fontsize=12)
    for lil_ax in ax:
        plt.setp(lil_ax.get_xticklabels(), fontsize=12)

    figs_axes.append((fig, ax, data_index))

for file, filepath in zip(h5_files, h5_filepaths):
    with h5py.File(filepath, 'r') as h5_file:
        data = h5_file.get(dataset_name)[()]
        x_axis_centre_index = data.shape[0]//2 # round down if the length is odd
        z_axis_centre_index = data.shape[1]//2 # round down if the length is odd
        
        x_axis_centre_trajectory = data[x_axis_centre_index]
        z_axis_indices = [
            0,
            z_axis_centre_index,
            data.shape[1] - 1
        ]

        for fig, ax, data_index in figs_axes:
            for counter, col in enumerate(ax):
                col.set_title('z[%d]' % z_axis_indices[counter], fontsize=18)
                data_slice = x_axis_centre_trajectory[z_axis_indices[counter]]
                s = np.linspace(0, len(data_slice), len(data_slice))
                x_data = data_slice.transpose()[data_index]
                col.plot(s, x_data, label=file, linewidth=1.0, alpha=0.7)

if job_type == 'shim':
    # plot the original genome too: it's contained in every shimmed genome so just
    # arbitrarily pick the 0th element in the list of h5 files
    with h5py.File(h5_filepaths[0], 'r') as h5_file:
        data = h5_file.get('id_trajectory_original')[()]
        x_axis_centre_index = data.shape[0]//2 # round down if the length is odd
        z_axis_centre_index = data.shape[1]//2 # round down if the length is odd

        x_axis_centre_trajectory = data[x_axis_centre_index]
        z_axis_indices = [
            0,
            z_axis_centre_index,
            data.shape[1] - 1
        ]

        for fig, ax, data_index in figs_axes:
            for counter, col in enumerate(ax):
                data_slice = x_axis_centre_trajectory[z_axis_indices[counter]]
                s = np.linspace(0, len(data_slice), len(data_slice))
                x_data = data_slice.transpose()[data_index]
                col.plot(s, x_data, label='original genome', linewidth=1.0, alpha=0.7)
                
handles, labels = col.get_legend_handles_labels()
# put the legend below the second figure
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.51, -0.18), bbox_transform=plt.gcf().transFigure, title='Genomes', title_fontsize=14, fontsize=14)
plt.show()

# Plot z trajectory and angle

In [ ]:
Z_HARDCODED_DATA_INDICES = [1, 3]
Z_HARDCODED_TITLES = ['Z trajectory\n', 'Z angle\n']

figs_axes = []
for data_index, plot_title in zip(Z_HARDCODED_DATA_INDICES, Z_HARDCODED_TITLES):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 10), sharex=True, sharey=True)
    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.grid(False)
    plt.title(plot_title, fontsize=20)
    plt.xlabel('s', labelpad=10, fontsize=15)
    plt.ylabel('z', labelpad=30, fontsize=15)
    # this doesn't feel too good to have to check the data_index just to get scientific
    # notation for the y axis for the angle plots, but it works for now
    if data_index == 3:
        ax[0].ticklabel_format(axis='y', style='scientific', scilimits=(0,0))
    
    # as the y axis is shared, only the 0th subplot needs to be edited, ie, the left-most one
    plt.setp(ax[0].get_yticklabels(), fontsize=12)
    for lil_ax in ax:
        plt.setp(lil_ax.get_xticklabels(), fontsize=12)
        
    figs_axes.append((fig, ax, data_index))

for file, filepath in zip(h5_files, h5_filepaths):
    with h5py.File(filepath, 'r') as h5_file:
        data = h5_file.get(dataset_name)[()]
        x_axis_centre_index = data.shape[0]//2 # round down if the length is odd
        z_axis_centre_index = data.shape[1]//2 # round down if the length is odd
        
        x_axis_indices = [
            0,
            x_axis_centre_index,
            data.shape[0] - 1
        ]

        for fig, ax, data_index in figs_axes:
            for counter, col in enumerate(ax):
                col.set_title('x[%d]' % x_axis_indices[counter], fontsize=18)
                data_slice = data[x_axis_indices[counter]][z_axis_centre_index]
                s = np.linspace(0, len(data_slice), len(data_slice))
                z_data = data_slice.transpose()[data_index]
                col.plot(s, z_data, label=file, linewidth=1.0, alpha=0.7)

if job_type == 'shim':
    # plot the original genome too: it's contained in every shimmed genome so just
    # arbitrarily pick the 0th element in the list of h5 files
    with h5py.File(h5_filepaths[0], 'r') as h5_file:
        data = h5_file.get('id_trajectory_original')[()]
        x_axis_centre_index = data.shape[0]//2 # round down if the length is odd
        z_axis_centre_index = data.shape[1]//2 # round down if the length is odd

        x_axis_centre_trajectory = data[x_axis_centre_index]
        z_axis_indices = [
            0,
            z_axis_centre_index,
            data.shape[1] - 1
        ]

        for fig, ax, data_index in figs_axes:
            for counter, col in enumerate(ax):
                data_slice = x_axis_centre_trajectory[z_axis_indices[counter]]
                s = np.linspace(0, len(data_slice), len(data_slice))
                x_data = data_slice.transpose()[data_index]
                col.plot(s, x_data, label='original genome', linewidth=1.0, alpha=0.7)
                
handles, labels = col.get_legend_handles_labels()
# put the legend below the second figure
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.51, -0.18), bbox_transform=plt.gcf().transFigure, title='Genomes', title_fontsize=14, fontsize=14)
plt.show()